In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")
print("SparkSession created with reference name as 'spark'")
#spark.stop()

SparkSession created with reference name as 'spark'


In [2]:
df=spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("../../../data/SparkTheDefinitiveGuide/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



### Booleans

In [3]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
.select("InvoiceNo", "Description").show(5, True)

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
+---------+--------------------+
only showing top 5 rows



In [50]:
df.where("InvoiceNo = 536365").show(2,False)
df.where("InvoiceNo <> 536365").show(2,False)

+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN               |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows

+---------+---------+-------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description              |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+------

In [51]:
from pyspark.sql.functions import instr,col
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1 # instr = contains()
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)).where("isExpensive").show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+



In [52]:
from pyspark.sql.functions import expr
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")).where("isExpensive").show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+



### Numbers

In [53]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(col("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [54]:
df.selectExpr("CustomerId","(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [55]:
from pyspark.sql.functions import corr
df.stat.corr("Quantity", "UnitPrice")

-0.04112314436835551

In [56]:
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [57]:
from pyspark.sql.functions import round, bround
df.select(round(col("UnitPrice"), 1).alias("rounded"), col("UnitPrice")).show(2)

+-------+---------+
|rounded|UnitPrice|
+-------+---------+
|    2.6|     2.55|
|    3.4|     3.39|
+-------+---------+
only showing top 2 rows



In [58]:
# Rownum
from pyspark.sql.functions import monotonically_increasing_id
df.withColumn('RowNum',monotonically_increasing_id()).show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|RowNum|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|     0|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|     1|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|     2|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|     3|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|     4|
+---------+---------+--------------------+--------+-------------------+---------

### String

In [59]:
from pyspark.sql.functions import initcap , lower, upper
df.select(col("Description"),initcap(col("Description")), lower(col("Description")),upper (col("Description") )).show(2)

+--------------------+--------------------+--------------------+--------------------+
|         Description|initcap(Description)|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|White Hanging Hea...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| White Metal Lantern| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [60]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
ltrim(lit(" HELLO ")).alias("ltrim"),
rtrim(lit(" HELLO ")).alias("rtrim"),
trim(lit(" HELLO ")).alias("trim"),
lpad(lit("HELLO"), 10, " ").alias("lp"),
rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

+------+------+-----+----------+----------+
| ltrim| rtrim| trim|        lp|        rp|
+------+------+-----+----------+----------+
|HELLO | HELLO|HELLO|     HELLO|HELLO     |
|HELLO | HELLO|HELLO|     HELLO|HELLO     |
+------+------+-----+----------+----------+
only showing top 2 rows



##### Regex

In [61]:
from pyspark.sql.functions import regexp_replace  # Replace anything from the StringList with "COLOR"
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [64]:
from pyspark.sql.functions import translate   # replace L with 1, E with 3, T with 7 
df.select(translate(col("Description"), "LET", "137"),col("Description")).show(2) 

+--------------------------------+--------------------+
|translate(Description, LET, 137)|         Description|
+--------------------------------+--------------------+
|            WHI73 HANGING H3A...|WHITE HANGING HEA...|
|             WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+--------------------------------+--------------------+
only showing top 2 rows



In [65]:
from pyspark.sql.functions import regexp_extract  # pull first occurence 
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),col("Description")).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



In [68]:
# Contaics/ instr 
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite).select("Description", "hasSimpleColor").show(3, False)

+----------------------------------+--------------+
|Description                       |hasSimpleColor|
+----------------------------------+--------------+
|WHITE HANGING HEART T-LIGHT HOLDER|true          |
|WHITE METAL LANTERN               |true          |
|CREAM CUPID HEARTS COAT HANGER    |false         |
+----------------------------------+--------------+
only showing top 3 rows



### Dates and Timestamps

In [79]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())
dateDF.show(2)
#dateDF.createOrReplaceTempView("dateTable")

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2020-07-04|2020-07-04 23:30:...|
|  1|2020-07-04|2020-07-04 23:30:...|
+---+----------+--------------------+
only showing top 2 rows



In [80]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-06-29|        2020-07-09|
+------------------+------------------+
only showing top 1 row



In [82]:
dateDF.withColumn("week_ago", date_sub(col("today"), 7)).select(datediff(col("week_ago"), col("today"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



In [86]:
#to_date function allows you to
convert a string to a date
dateDF.select(
to_date(lit("2016-01-01")).alias("start"),
to_date(lit("2017-05-22")).alias("end"))\
.select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



#### Coalesce

In [6]:
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"), col("CustomerId"))).show(2)

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|                            17850|
+---------------------------------+
only showing top 2 rows

